In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.optimizers import Adam,SGD,Adadelta,RMSprop,Adagrad,Nadam,Adamax

In [3]:
cifar_100=tf.keras.datasets.cifar100


In [4]:
(x_train,y_train),(x_val,y_val)=cifar_100.load_data()

169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [5]:
y_train=tf.one_hot(y_train,
                   depth=y_train.max()+1,
                   dtype=tf.float64)
y_val=tf.one_hot(y_val,
                 depth=y_val.max()+1,
                 dtype=tf.float64)
y_train=tf.squeeze(y_train)
y_val=tf.squeeze(y_val)

In [19]:
def create_model(opt):
  model=tf.keras.models.Sequential([
      layers.Conv2D(16,(3,3),activation='relu',input_shape=(32,32,3),padding='same'),
      layers.Conv2D(32,(3,3),activation='relu',padding='same'),
      layers.Conv2D(64,(3,3),activation='relu',padding='same'),
      layers.MaxPooling2D(2,2),
      layers.Conv2D(128,(3,3),activation='relu',padding='same'),
      layers.Flatten(),
      layers.Dense(256,activation='relu'),
      layers.BatchNormalization(),
      layers.Dense(256,activation='relu'),
      layers.Dropout(0.3),
      layers.BatchNormalization(),
      layers.Dense(100,activation='softmax')


  ])
  return model

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
def create_callbacks(opt):
  checkpoints_path='checkpoints/optimizers_best_'+opt+'.h5'
  callbacks=[
      EarlyStopping(monitor='val_accuracy',patience=3,mode='max',verbose=1),
      ModelCheckpoint(checkpoints_path,monitor='val_accuracy',save_best_only=True,mode='max',verbose=1)
  ]
  return callbacks

In [22]:
opts=dict({
    'sgd':SGD(),
    'sgd-001':SGD(learning_rate=0.0001,decay=0.00001),
    'adam':Adam(),
    'adadelta':Adadelta(),
    'rmsprop':RMSprop(),
    'rmsprop-0001':RMSprop(learning_rate=0.0001),
    'nadam':Nadam(),
    'adamax':Adamax()
})

In [23]:
batch_size=64
n_epochs=5
results=[]
for opt in opts:
  print("*****now model has taken"+opt+"as optimizer****")
  model=create_model(opt)
  callbacks=create_callbacks(opt)
  model.compile(loss='mse',optimizer=opts[opt],metrics=['accuracy'])
  hist=model.fit(x_train,y_train,batch_size=batch_size,epochs=n_epochs,validation_data=(x_val,y_val),verbose=1,callbacks=callbacks)
  best_epoch=np.argmax(hist.history['val_accuracy'])
  best_acc=hist.history['val_accuracy'][best_epoch]
  best_model=create_model(opt)
  best_model.load_weights('checkpoints/optimizers_best_'+opt+'.h5')
  best_model.compile(loss='mse',optimizer=opts[opt],metrics=['accuracy'])
  print("now model gets evaluated")
  score=best_model.evaluate(x_val,y_val,verbose=1)
  results.append([opt,best_epoch,best_acc,score[1]])
results_df=pd.DataFrame(results,columns=['optimizer','best_epoch','best validation accuracy','verbose'])
print("\n**results summary***")
print(results_df)

*****now model has takensgdas optimizer****
Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0099 - loss: 0.0102
Epoch 1: val_accuracy improved from -inf to 0.00850, saving model to checkpoints/optimizers_best_sgd.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - accuracy: 0.0099 - loss: 0.0102 - val_accuracy: 0.0085 - val_loss: 0.0101
Epoch 2/5
779/782 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0105 - loss: 0.0102
Epoch 2: val_accuracy improved from 0.00850 to 0.00940, saving model to checkpoints/optimizers_best_sgd.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.0105 - loss: 0.0102 - val_accuracy: 0.0094 - val_loss: 0.0101
Epoch 3/5
776/782 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0106 - loss: 0.0102
Epoch 3: val_accuracy improved from 0.00940 to 0.01180, saving model to checkpoints/optimizers_best_sgd.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.0106 - loss: 0.0102 - val_accuracy: 0.0118 - val_loss: 0.0101
Epoch 4/5
781/782 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0111 - loss: 0.0101
Epoch 4: val_accuracy improved from 0.01180 to 0.01200, saving model to checkpoints/optimizers_best_sgd.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.0111 - loss: 0.0101 - val_accuracy: 0.0120 - val_loss: 0.0100
Epoch 5/5
776/782 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0118 - loss: 0.0101
Epoch 5: val_accuracy improved from 0.01200 to 0.01260, saving model to checkpoints/optimizers_best_sgd.h5


...

782/782 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - accuracy: 0.0368 - loss: 0.0100 - val_accuracy: 0.0952 - val_loss: 0.0097
Epoch 2/5
780/782 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1132 - loss: 0.0096
Epoch 2: val_accuracy improved from 0.09520 to 0.14980, saving model to checkpoints/optimizers_best_adamax.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.1133 - loss: 0.0096 - val_accuracy: 0.1498 - val_loss: 0.0094
Epoch 3/5
777/782 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1735 - loss: 0.0092
Epoch 3: val_accuracy improved from 0.14980 to 0.18550, saving model to checkpoints/optimizers_best_adamax.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.1735 - loss: 0.0092 - val_accuracy: 0.1855 - val_loss: 0.0091
Epoch 4/5
780/782 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2301 - loss: 0.0088
Epoch 4: val_accuracy improved from 0.18550 to 0.25410, saving model to checkpoints/optimizers_best_adamax.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.2301 - loss: 0.0088 - val_accuracy: 0.2541 - val_loss: 0.0086
Epoch 5/5
777/782 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2882 - loss: 0.0083
Epoch 5: val_accuracy improved from 0.25410 to 0.28760, saving model to checkpoints/optimizers_best_adamax.h5


782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.2883 - loss: 0.0083 - val_accuracy: 0.2876 - val_loss: 0.0083
now model gets evaluated
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2867 - loss: 0.0083

**results summary***
      optimizer  best_epoch  best validation accuracy  verbose
0           sgd           4                    0.0126   0.0126
1       sgd-001           3                    0.0107   0.0107
2          adam           4                    0.2498   0.2498
3      adadelta           3                    0.0107   0.0107
4       rmsprop           4                    0.1920   0.1920
5  rmsprop-0001           4                    0.1055   0.1055
6         nadam           4                    0.2364   0.2364
7        adamax           4                    0.2876   0.2876
